In [10]:
import pandas as pd
from tqdm import tqdm
annual_report_info=pd.read_csv('../data/train/annual_report_info.csv')#企业的年报基本信息

In [11]:
count, shapes = annual_report_info.shape
#空值大于0.7的列都删除掉
annual_report_info_clean=annual_report_info.dropna(thresh=annual_report_info.shape[0]*0.7,how='all',axis=1)

In [12]:
annual_report_info_clean.nunique()

id               8937
ANCHEYEAR           4
STATE               2
EMPNUM            270
EMPNUMSIGN          2
BUSSTNAME           4
COLGRANUM          43
RETSOLNUM          14
DISPERNUM           5
UNENUM             35
COLEMPLNUM         94
RETEMPLNUM         26
DISEMPLNUM         16
UNEEMPLNUM         72
WEBSITSIGN          2
FORINVESTSIGN       2
PUBSTATE            3
dtype: int64

id:企业唯一标识, ANCHEYEAR:年度, STATE:状态, FUNDAM:资金数额, MEMNUM:成员人数, FARNUM:农民人数, ANNNEWMEMNUM:本年度新增成员人数, ANNREDMEMNUM:本年度退出成员人数, EMPNUM:从业人数, EMPNUMSIGN:从业人数是否公示, BUSSTNAME:经营状态名称, COLGRANUM:其中高校毕业生人数经营者, RETSOLNUM:其中退役士兵人数经营者, DISPERNUM:其中残疾人人数经营者, UNENUM:其中下岗失业人数经营者, COLEMPLNUM:其中高校毕业生人数雇员, RETEMPLNUM:其中退役士兵人数雇员, DISEMPLNUM:其中残疾人人数雇员, UNEEMPLNUM:其中下岗失业人数雇员, WEBSITSIGN:是否有网站标志, FORINVESTSIGN:是否有对外投资企业标志, STOCKTRANSIGN:有限责任公司本年度是否发生股东股权转让标志, PUBSTATE:公示状态：1 全部公示，2部分公示,3全部不公示

In [13]:
# 人数的信息
num_cols = ['EMPNUM','COLGRANUM','RETSOLNUM','DISPERNUM','UNENUM','COLEMPLNUM','RETEMPLNUM','DISEMPLNUM','UNEEMPLNUM']
annual_report_info_clean[num_cols] = annual_report_info_clean[num_cols].fillna(0.0,axis = 1)
annual_report_info_clean[num_cols].head()


,EMPNUM,COLGRANUM,RETSOLNUM,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM
0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
for num_col in tqdm(num_cols):
    num  = (annual_report_info_clean[num_col].values == 0).sum()
    if num*1.0/count >= 0.8:
        annual_report_info_clean.drop(num_col,axis = 1,inplace=True)

100%|██████████| 9/9 [00:00<00:00, 408.59it/s]


In [15]:
num_cols = list(set(num_cols) & set(annual_report_info_clean.columns))
annual_report_info_clean[num_cols].head()

,COLEMPLNUM,EMPNUM,COLGRANUM
0,0.0,10.0,0.0
1,0.0,2.0,0.0
2,1.0,4.0,3.0
3,2.0,3.0,1.0
4,0.0,10.0,0.0


In [16]:
# category 类别进行编码
cat_cols = ['STATE','EMPNUMSIGN','BUSSTNAME','FORINVESTSIGN','WEBSITSIGN','FORINVESTSIGN','PUBSTATE']
# 类别信息 取最新非nan数据
# 雇佣人数，取均值，增量，
grouped = annual_report_info_clean.sort_values(by='ANCHEYEAR',ascending= False).groupby('id')
clean_infos = []
for name, group_info in tqdm(grouped):
     clean_info = {'id':name}
     clean_info['ANCHEYEAR'] = group_info['ANCHEYEAR'].values[-1]
     clean_info['REPORT_NUM'] = len(group_info) 
     clean_info['HAS_REPORT'] = 1.0
     for cat_col in cat_cols:
         clean_info[cat_col] = group_info[cat_col].values[-1]
     for num_col in num_cols:
         clean_info[f'{num_col}'] = group_info[num_col].values[-1]
         clean_info[f'{num_col}_MEAN'] = group_info[num_col].values.mean()
         clean_info[f'{num_col}_ADD'] = 0.0
         if len(group_info) > 1:
             clean_info[f'{num_col}_ADD'] = group_info[num_col].values[-1] - group_info[num_col].values[-2]     
     clean_infos.append(clean_info)
clean_infos = pd.DataFrame(clean_infos)

100%|██████████| 8937/8937 [00:04<00:00, 1865.97it/s]


In [17]:
clean_infos.to_csv('./features/annual_report_info_feature.csv',index = False)

In [18]:
clean_infos.describe()

,ANCHEYEAR,REPORT_NUM,HAS_REPORT,STATE,EMPNUMSIGN,FORINVESTSIGN,WEBSITSIGN,PUBSTATE,COLEMPLNUM,COLEMPLNUM_MEAN,COLEMPLNUM_ADD,EMPNUM,EMPNUM_MEAN,EMPNUM_ADD,COLGRANUM,COLGRANUM_MEAN,COLGRANUM_ADD
count,8937.000000,8937.000000,8937.0,8934.000000,6218.000000,6117.000000,8916.000000,8929.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000
mean,2016.271232,2.523218,1.0,1.999440,1.877292,1.957822,1.972185,2.794714,1.263847,1.217362,0.001567,8.375518,9.061756,-0.912946,0.826340,0.707435,0.155533
std,1.221567,1.248877,0.0,0.023652,0.328128,0.201010,0.164452,0.499856,6.922546,6.300454,6.832944,59.917441,101.814570,101.120169,5.956445,2.930997,5.905040
min,2015.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,-492.000000,0.000000,0.000000,-8808.000000,0.000000,0.000000,-80.000000
25%,2015.000000,1.000000,1.0,2.000000,2.000000,2.000000,2.000000,3.000000,0.000000,0.000000,0.000000,2.000000,1.500000,0.000000,0.000000,0.000000,0.000000
50%,2016.000000,2.000000,1.0,2.000000,2.000000,2.000000,2.000000,3.000000,0.000000,0.000000,0.000000,2.000000,2.500000,0.000000,0.000000,0.000000,0.000000
75%,2017.000000,4.000000,1.0,2.000000,2.000000,2.000000,2.000000,3.000000,1.000000,1.000000,0.000000,5.000000,5.000000,0.000000,1.000000,1.000000,0.000000
max,2018.000000,8.000000,1.0,2.000000,2.000000,2.000000,2.000000,3.000000,382.000000,309.500000,177.000000,2587.000000,8050.750000,607.000000,408.000000,177.000000,408.000000
